In [1]:
from netgen.occ import *
from ngsolve import *
from ngsolve.internal import *
from xfem import *
from xfem.lsetcurv import *
from math import pi,e
from numpy import linspace

ngsglobals.msg_level = 2

importing ngsxfem-2.1.2504


In [2]:
# physical parameters for Biot
mu  = 10
lam = 10
alpha = 1
# alpha = 0
K = 1 # k^-1
nu = 1
# s0 = 10
s0 = 1e-2

def Stress(strain):
    return 2*mu*strain + lam*Trace(strain)*Id(3)

In [3]:
# parameters 


# DG space order
order_eta = 2
order_u = 2
order_p = 1


In [4]:
# penalty parameters
beta_eta = 400
beta_u = 200
# ghost penalty parameters
gamma_s = 0
gamma_u = 0
gamma_p = 0.1
gamma_m = 0
tau_p = 0.1

In [5]:
# 定义解析解
eta_x = sin(pi*x) * sin(pi*y) * sin(pi*z)
eta_y = cos(pi*x) * sin(pi*y) * sin(pi*z)
eta_z = sin(pi*x) * cos(pi*y) * sin(pi*z)

u_x = sin(pi*x) * cos(pi*y) * sin(pi*z)
u_y = -cos(pi*x) * sin(pi*y) * sin(pi*z)
u_z = sin(pi*x) * sin(pi*y) * cos(pi*z)

exact_eta = CF((eta_x, eta_y, eta_z))
exact_u = CF((u_x, u_y, u_z))
exact_p = sin(pi*x)*sin(pi*y)*sin(pi*z)


# strain tensor
epsilon_xx = eta_x.Diff(x)
epsilon_yy = eta_y.Diff(y) 
epsilon_zz = eta_z.Diff(z) 
epsilon_xy = 0.5*(eta_x.Diff(y) +  eta_y.Diff(x))
epsilon_yz = 0.5*(eta_y.Diff(z) +  eta_z.Diff(y))
epsilon_xz = 0.5*(eta_x.Diff(z) +  eta_z.Diff(x))

# total stress tensor
sigma_xx = lam*(epsilon_xx + epsilon_yy + epsilon_zz) + 2*mu*epsilon_xx - alpha*exact_p
sigma_yy = lam*(epsilon_xx + epsilon_yy + epsilon_zz) + 2*mu*epsilon_yy - alpha*exact_p
sigma_zz = lam*(epsilon_xx + epsilon_yy + epsilon_zz) + 2*mu*epsilon_zz - alpha*exact_p
sigma_xy = 2*mu*epsilon_xy
sigma_yz = 2*mu*epsilon_yz
sigma_xz = 2*mu*epsilon_xz


# 右端项 f_x, f_y
f_x = - (sigma_xx.Diff(x) + sigma_xy.Diff(y) + sigma_xz.Diff(z))
f_y = - (sigma_xy.Diff(x) + sigma_yy.Diff(y) + sigma_yz.Diff(z))
f_z = - (sigma_xz.Diff(x) + sigma_yz.Diff(y) + sigma_zz.Diff(z))

fm_x = K*u_x-nu*(u_x.Diff(x).Diff(x)+u_x.Diff(y).Diff(y)+u_x.Diff(z).Diff(z)) + exact_p.Diff(x)
fm_y = K*u_y-nu*(u_y.Diff(x).Diff(x)+u_y.Diff(y).Diff(y)+u_y.Diff(z).Diff(z)) + exact_p.Diff(y)
fm_z = K*u_z-nu*(u_z.Diff(x).Diff(x)+u_z.Diff(y).Diff(y)+u_z.Diff(z).Diff(z)) + exact_p.Diff(z)

# 向量形式
fe = CF((f_x, f_y, f_z))
fm = CF((fm_x, fm_y, fm_z))
fp = s0*exact_p+alpha*(eta_x.Diff(x)+eta_y.Diff(y)+eta_z.Diff(z)) + u_x.Diff(x) + u_y.Diff(y) + u_z.Diff(z)

etaD = exact_eta
uD = exact_u
pD = exact_p

In [6]:
# Set level set function
# levelset = x
levelset = sqrt(x**2 + y**2) - 0.5
# levelset = sqrt(x**2 + y**2 + z**2) - 0.5
# levelset = IfPos((1+z)*(1-z),sqrt(x**2 + y**2) - 0.5,1)

In [7]:
# Construct background mesh
# Geometry and Mesh
# Quadrilateral (or simplicial mesh)
quad_mesh = True
# Mesh diameter
h0 = 1/8

box = Box((-1,-1,-1/4), (1,1,1/4)) 
mesh = Mesh(OCCGeometry(box).GenerateMesh(maxh=h0))
Draw(mesh)

 Face 1 / 6 (parameter space projection)
 Face 2 / 6 (parameter space projection)
 Face 3 / 6 (parameter space projection)
 Face 4 / 6 (parameter space projection)
 Face 5 / 6 (parameter space projection)
 Face 6 / 6 (parameter space projection)
 Delaunay meshing
 Volume Optimization


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [8]:
# Higher order level set approximation
# lsetmeshadap = LevelSetMeshAdaptation(mesh, order=2, threshold=0.1,
#                                       discontinuous_qn=True)
# deformation = lsetmeshadap.CalcDeformation(levelset)
# lsetp1 = lsetmeshadap.lset_p1

lsetp1 = GridFunction(H1(mesh,order=1))
InterpolateToP1(levelset,lsetp1)
DrawDC(lsetp1,-1,1,mesh,'lsetp1')

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [9]:
# Element, facet and dof marking w.r.t. boundary approximation with lsetp1:
ci = CutInfo(mesh, lsetp1)
hasneg = ci.GetElementsOfType(HASNEG)
hasif = ci.GetElementsOfType(IF)
Draw(BitArrayCF(hasneg),mesh)
# Draw(BitArrayCF(hasif),mesh)

# facets used for stabilization:
ba_facets = GetFacetsWithNeighborTypes(mesh, a=hasneg, b=hasif)
ba_surround_facets = GetElementsWithNeighborFacets(mesh,ba_facets)
interior_facets = GetFacetsWithNeighborTypes(mesh, a=hasneg, b=hasneg)
in_surround_facets = GetElementsWithNeighborFacets(mesh,interior_facets)
# Draw(BitArrayCF(ba_surround_facets), mesh, "surrounding_facets") 
# Draw(BitArrayCF(in_surround_facets), mesh, "surrounding_facets") 




WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

In [ ]:
print(help(ci))

In [ ]:
kappaminus = CutRatioGF(ci)
kappaminus_values = kappaminus.vec.FV().NumPy()

# 找出非零值中的最小值
# 这里需要过滤掉那些没有被切割的单元的值（通常是0）
# min_value = 1.0 # 初始化一个比任何比率都大的值
# for value in kappaminus_values:
#     if value > 0 and value < min_value:
#         min_value = value

# print(f"所有被切割单元中，比值的最小值为: {min_value}")

# Or
positive_values = [v for v in kappaminus_values if v > 0]
if positive_values:
    min_value_pythonic = min(positive_values)
    print(f"所有被切割单元中，比值的最小值为 (Pythonic): {min_value_pythonic}")
else:
    print("没有找到被切割的单元。")

In [ ]:
#Unfitted DG spaces
Ehbase = VectorL2(mesh, order=order_eta, dirichlet=[], dgjumps=True) # space for displacement
Uhbase = VectorL2(mesh, order=order_u, dirichlet=[], dgjumps=True) # space for velocity
# Phbase = L2(mesh, order=order_p, dirichlet=[], dgjumps=True) # space for pressure 
Phbase = H1(mesh, order=order_p, dirichlet=[], dgjumps=True) # space for pressure 
E = Compress(Ehbase, GetDofsOfElements(Ehbase, ci.GetElementsOfType(HASNEG)))
U = Compress(Uhbase, GetDofsOfElements(Uhbase, ci.GetElementsOfType(HASNEG)))
P = Compress(Phbase, GetDofsOfElements(Phbase, ci.GetElementsOfType(HASNEG)))
fes = E*U*P
(eta,u,p), (kxi,v,q) = fes.TnT()

In [ ]:
# Define the jumps and the averages
# n = Normalize(grad(lsetp1)) # outer normal vector on the boundary
n = 1.0 / Norm(grad(lsetp1)) * grad(lsetp1)
ne = specialcf.normal(mesh.dim) # normal vectors on faces
h = specialcf.mesh_size  

strain_eta = Sym(Grad(eta))
strain_kxi = Sym(Grad(kxi))
mean_stress_eta = 0.5*(Stress(Sym(Grad(eta)))+Stress(Sym(Grad(eta.Other()))))*ne
mean_stress_kxi = 0.5*(Stress(Sym(Grad(kxi)))+Stress(Sym(Grad(kxi.Other()))))*ne
jump_eta = eta - eta.Other()
jump_kxi = kxi - kxi.Other()


jump_p = p - p.Other()
jump_q = q - q.Other()
mean_p = 0.5*(p + p.Other())
mean_q = 0.5*(q + q.Other())


mean_dudn = 0.5*(Grad(u)+Grad(u.Other()))*ne
mean_dvdn = 0.5*(Grad(v)+Grad(v.Other()))*ne
jump_u = u - u.Other()
jump_v = v - v.Other()
 

In [ ]:
# integration domains:
domega = dCut(lsetp1, NEG, definedonelements=hasneg, deformation=deformation)
dk = dCut(lsetp1, NEG, skeleton=True, definedonelements=interior_facets,
          deformation=deformation)
ds = dCut(lsetp1, IF, definedonelements=hasif, deformation=deformation)
dw = dFacetPatch(definedonelements=ba_facets, deformation=deformation)
 # = dFacetPatch(definedonelements=interior_facets, deformation=deformation)

In [ ]:
Ah = BilinearForm(fes)
# Ae
Ah += 2*mu*InnerProduct(strain_eta,strain_kxi)*domega + lam*div(eta)*div(kxi)*domega \
        - (InnerProduct(mean_stress_eta,jump_kxi) + InnerProduct(mean_stress_kxi,jump_eta) - beta_eta/h*InnerProduct(jump_eta,jump_kxi))*dk \
        - (InnerProduct(Stress(Sym(Grad(eta)))*n,kxi) + InnerProduct(Stress(Sym(Grad(kxi)))*n,eta) - beta_eta/h*InnerProduct(eta,kxi))*ds
# order=1 i_s 
Ah += gamma_s * h * InnerProduct(Grad(eta)*ne - Grad(eta.Other())*ne,Grad(kxi)*ne - Grad(kxi.Other())*ne) * dw

# Be
Ah += -alpha*(div(kxi)*p*domega - mean_p*jump_kxi*ne*dk - p*kxi*n*ds)

              
# Am
Ah += nu*InnerProduct(Grad(u),Grad(v))*domega \
        - nu*(InnerProduct(mean_dudn,jump_v) + InnerProduct(mean_dvdn,jump_u) - beta_u/h*InnerProduct(jump_u,jump_v))*dk \
        - nu*(InnerProduct(Grad(u)*n,v) + InnerProduct(Grad(v)*n,u) - beta_u/h*InnerProduct(u,v))*ds\
        + K*InnerProduct(u,v)*domega
# ghost penalty for velocity
Ah += gamma_u * h * InnerProduct(Grad(u)*ne - Grad(u.Other())*ne,Grad(v)*ne - Grad(v.Other())*ne) * dw


 # Bm 
Ah += -div(v)*p*domega + mean_p*jump_v*ne*dk + p*v*n*ds

# -Bm
Ah += div(u)*q*domega - mean_q*jump_u*ne*dk - q*u*n*ds

# order=1 i_p 
Ah += gamma_p * (h**3) * (grad(p)*ne - grad(p.Other())*ne)*(grad(q)*ne - grad(q.Other())*ne) * dw

# M
Ah += s0*p*q*domega + gamma_m*(h**3)*(grad(p)*ne - grad(p.Other())*ne)*(grad(q)*ne - grad(q.Other())*ne) * dw

# -Be
Ah += alpha*(div(eta)*q*domega - mean_q*jump_eta*ne*dk - q*eta*n*ds)

# stabilization
Ah += tau_p*h*jump_p*jump_q*dk
Ah.Assemble()

In [ ]:
# Ah = BilinearForm(fes)
# # Ae
# Ah += 2*mu*InnerProduct(strain_eta,strain_kxi)*domega + lam*div(eta)*div(kxi)*domega \
#         - (InnerProduct(mean_stress_eta,jump_kxi) + InnerProduct(mean_stress_kxi,jump_eta) - beta_eta/h*InnerProduct(jump_eta,jump_kxi))*dx(skeleton=True) \
#         - (InnerProduct(Stress(Sym(Grad(eta)))*n,kxi) + InnerProduct(Stress(Sym(Grad(kxi)))*n,eta) - beta_eta/h*InnerProduct(eta,kxi))*ds
# # order=1 i_s 
# Ah += gamma_s * h * InnerProduct(Grad(eta)*ne - Grad(eta.Other())*ne,Grad(kxi)*ne - Grad(kxi.Other())*ne) * dw

# # Be
# Ah += -alpha*(div(kxi)*p*domega - mean_p*jump_kxi*ne*dx(skeleton=True) - p*kxi*n*ds)

              
# # Am
# Ah += nu*InnerProduct(Grad(u),Grad(v))*domega \
#         - nu*(InnerProduct(mean_dudn,jump_v) + InnerProduct(mean_dvdn,jump_u) - beta_u/h*InnerProduct(jump_u,jump_v))*dx(skeleton=True) \
#         - nu*(InnerProduct(Grad(u)*n,v) + InnerProduct(Grad(v)*n,u) - beta_u/h*InnerProduct(u,v))*ds\
#         + K*InnerProduct(u,v)*domega

#  # Bm 
# Ah += -div(v)*p*domega + mean_p*jump_v*ne*dx(skeleton=True) + p*v*n*ds

# # -Bm
# Ah += div(u)*q*domega - mean_q*jump_u*ne*dx(skeleton=True) - q*u*n*ds

# # order=1 i_p 
# Ah += gamma_p * h * (grad(p)*ne - grad(p.Other())*ne)*(grad(q)*ne - grad(q.Other())*ne) * dw

# # M
# Ah += s0*p*q*domega + gamma_m*(h**3)*(grad(p)*ne - grad(p.Other())*ne)*(grad(q)*ne - grad(q.Other())*ne) * dw

# # -Be
# Ah += alpha*(div(eta)*q*domega - mean_q*jump_eta*ne*dx(skeleton=True) - q*eta*n*ds)
# Ah.Assemble()

In [ ]:
# r.h.s
lh = LinearForm(fes) 
lh += fe*kxi*domega - InnerProduct(etaD,Stress(Sym(Grad(kxi)))*n)*ds + beta_eta/h*etaD*kxi*ds
lh += fm*v*domega - nu*InnerProduct(uD,Grad(v)*n)*ds + nu*beta_u/h*uD*v*ds
lh += fp*q*domega - alpha*q*etaD*n*ds - q*uD*n*ds
lh.Assemble()

In [ ]:
gfu = GridFunction(fes)
gfu.vec.data = Ah.mat.Inverse() * lh.vec

error_eta = sqrt(Integrate((gfu.components[0] - exact_eta)**2* domega, mesh))
error_u = sqrt(Integrate((gfu.components[1] - exact_u)**2 * domega, mesh))
error_p = sqrt(Integrate((gfu.components[2] - exact_p)**2 * domega, mesh))

print(error_eta)
print(error_u)
print(error_p)

In [ ]:
mask = IfPos(levelset,0,1)

In [ ]:
Vh_eta = VectorH1(mesh, order=order_eta)
eta_cont = GridFunction(Vh_eta)
eta_cont.Set(gfu.components[0])     # 将 DG 解作为 CF 插值到 H1
Draw(mask*eta_cont, mesh)
Draw(mask*exact_eta, mesh)

In [ ]:
Draw(mask*gfu.components[1],mesh)
Draw(mask*exact_u,mesh)

In [ ]:
Draw(mask*gfu.components[2],mesh)
# Draw(mask*gfu.components[2],mesh,min=-1, max=1)
Draw(mask*exact_p,mesh)

In [ ]:
# import numpy as np
# import scipy.sparse as sp

# rows,cols,vals = Ah.mat.COO()
# A = sp.csr_matrix((vals,(rows,cols)))
# conds = np.linalg.cond(A.todense())

In [ ]:
# print(conds)